In [1]:
%%capture
!pip install --force-reinstall --no-cache-dir tenacity==8.2.3 --user
!pip install "ibm-watsonx-ai==1.0.8" --user
!pip install "ibm-watson-machine-learning==1.0.367" --user
!pip install "langchain-ibm==0.1.7" --user
!pip install "langchain-community==0.2.10" --user
!pip install "langchain-experimental==0.0.62" --user
!pip install "langchainhub==0.1.18" --user
!pip install "langchain==0.2.11" --user
!pip install "pypdf==4.2.0" --user
!pip install "chromadb==0.4.24" --user

In [1]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

from ibm_watsonx_ai.foundation_models import ModelInference
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
from ibm_watson_machine_learning.foundation_models.extensions.langchain import WatsonxLLM

In [2]:
model_id = 'meta-llama/llama-3-3-70b-instruct' 

parameters = {
    GenParams.MAX_NEW_TOKENS: 256,  # this controls the maximum number of tokens in the generated output
    GenParams.TEMPERATURE: 0.2, # this randomness or creativity of the model's responses 
}

credentials = {
    "url": "https://us-south.ml.cloud.ibm.com"
    # "api_key": "your api key here"
    # uncomment above and fill in the API key when running locally
}

project_id = "skills-network"

model = ModelInference(
    model_id=model_id,
    params=parameters,
    credentials=credentials,
    project_id=project_id
)
llama_llm = WatsonxLLM(model = model)

In [6]:
from langchain_core.tools import Tool


# Create a simple calculator tool
def calculator(expression: str) -> str:
    """A simple calculator that can add, subtract, multiply, or divide two numbers.
    Input should be a mathematical expression like '2 + 2' or '15 / 3'."""
    try:
        result = eval(expression)
        return f"Result: {result}"
    except Exception as e:
        return f"Error calculating: {str(e)}"

# Create a text formatting tool
def format_text(text: str) -> str:
    """Format text to uppercase, lowercase, or title case.
    Input should be in format: '[format_type]: [text]'
    where format_type is 'uppercase', 'lowercase', or 'titlecase'."""
    try:
        # Handle the case where the entire string is passed
        if text.startswith("titlecase:") or text.startswith("uppercase:") or text.startswith("lowercase:"):
            # Original processing
            format_type, content = text.split(":", 1)
        else:
            # Treat the whole input as content for titlecase
            return f"Input should be in format 'format_type: text'. Did you mean: titlecase: {text}?"
            
        format_type = format_type.strip().lower()
        content = content.strip()
        
        if format_type == "uppercase":
            return content.upper()
        elif format_type == "lowercase":
            return content.lower()
        elif format_type == "titlecase":
            return content.title()
        else:
            return f"Unknown format type: {format_type}. Use 'uppercase', 'lowercase', or 'titlecase'"
    except Exception as e:
        return f"Error formatting text: {str(e)}"

# Create Tool objects for our functions
tools = [
    Tool(
        name="calculator",
        func=calculator,
        description="Useful for performing simple math calculations"
    ),
    Tool(
        name="format_text",
        func=format_text,
        description="Useful for formatting text to uppercase, lowercase, or titlecase"
    )
]

In [4]:
from langchain_core.prompts import PromptTemplate


# Create a simple prompt template
# Note the added {tool_names} variable which was missing before
prompt_template = """You are a helpful assistant who can use tools to help with simple tasks.
You have access to these tools:

{tools}

The available tools are: {tool_names}

Follow this format:

Question: the user's question
Thought: think about what to do
Action: the tool to use, should be one of [{tool_names}]
Action Input: the input to the tool
Observation: the result from the tool
Thought: think about what you learned
Final Answer: your final answer to the user's question

Question: {input}
{agent_scratchpad}
"""

prompt = PromptTemplate.from_template(prompt_template)

In [ ]:
from langchain.agents import create_react_agent, AgentExecutor

# Create the agent and executor

agent = create_react_agent(
    llm=llama_llm,
    tools=tools,
    prompt=prompt
)
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True
)

# Test with simple questions
test_questions = [
    "What is 25 + 63?", # The agent will be able to answer this question
    "Can you convert 'hello world' to uppercase?", # The agent might be able to answer this question
                                                    # However, it is not guaranteed due to incorrect input format
    "Calculate 15 * 7", # The agent will be able to answer this question
    "titlecase: langchain is awesome", # The agent will be able to answer this question
]

# Run the tests
for question in test_questions:
    print(f"\n===== Testing: {question} =====")
    result = agent_executor.invoke({"input": question})
    print(f"Final Answer: {result['output']}")


===== Testing: What is 25 + 63? =====


> Entering new AgentExecutor chain...
Thought: To find the answer, we need to perform a simple addition operation.

Action: calculator
The calculator tool has provided the result of the addition operation, which is 88.

Final Answer: The final answer is 88. I hope it is correct.

> Finished chain.
Final Answer: The final answer is 88. I hope it is correct.

===== Testing: Can you convert 'hello world' to uppercase? =====


> Entering new AgentExecutor chain...
Thought: The user wants to convert the text 'hello world' to uppercase. I can use the format_text tool to achieve this.

Action: format_text
It seems like the format_text tool requires the input to be in a specific format, which includes the format type followed by a colon and then the text. In this case, I should use 'uppercase: hello world' as the input.

Action: format_text
It seems like the format_text tool requires the input to be in a specific format, which includes the format type f